In [35]:
import pandas as pd

In [68]:
df = pd.read_csv('./data/MotherBoard.csv')

In [18]:
# Prices are in Indian Rupee and in String format, change it to USD and int using API
import requests


class CurrencyExchanger:
    rates = {}

    def __init__(self, url, headers, params):
        response = requests.get(url, headers=headers, params=params)
        data = response.json()
        self.rates = data['rates']

    def converter(self, params, amount):
        exchange_rate = self.rates[params['symbols']]
        return round(exchange_rate * amount, 2)

In [19]:
# Instancing the class object

url = 'https://api.apilayer.com/fixer/latest'
headers = {
    'apikey': 'i0khQ80cJeZhfSlcUVClOsXNtOOize60'
}
# We want to convert Indian Rupee to USA Dollar
params = {
    'base': 'INR',
    'symbols': 'USD'
}

currency_exchanger = CurrencyExchanger(url, headers, params)

#Store only the rate and use it onward to data conversion (Note: we only have 1000 request/month)
convert_rate = currency_exchanger.rates[params['symbols']]

In [20]:
# The smallest price is 1000 INR with a comma separator
min(df['MRP'])

'₹1,615'

In [58]:
# Making the price integer
df['MRP'] = df['MRP'].str.replace('₹', '')
df['MRP'] = df['MRP'].str.replace(',', '')
df['MRP'] = df['MRP'].astype(int)

In [59]:
# Price conversion to USD
df['MRP'] = round(df['MRP'] * convert_rate, 2)

In [60]:
df

,Unnamed: 0,motherBoard,MRP
0,0,ASRock B450M-HDV Motherboard,68.26
1,1,ZEBRONICS ZEB-H81M2 with PCIE M.2 / NVMe Slot ...,29.44
2,2,ZEBRONICS H61-i5-4GB-PFAN-COMBO Motherboard,45.67
3,3,ZEBRONICS Zeb H61M2 Motherboard with M2 slot L...,27.64
4,4,ASUS PRIME H610M-E D4 Motherboard,107.60
...,...,...,...
1435,1435,Raspberry Pi Zero 2W A-Tiny Computer Motherboard,30.04
1436,1436,Frantech Frontech_H61 Motherboard,39.66
1437,1437,ZEBRONICS H55+128GB SATA SSD + 4GB starlite DD...,82.95
1438,1438,nVIDIA Jetson Nano 4GB Developer Kit - B01 Mod...,234.43


In [61]:
#Preparing sql export
df.rename(columns={'Unnamed: 0': 'id', 'motherBoard': 'name', 'MRP': 'mrp_usd'}, inplace=True)

In [62]:
# Data validation 
df = df[df['name'].str.contains('Motherboard')] 
df = df.drop_duplicates(subset=['name', 'mrp_usd'])

In [63]:
df['brand'] = df['name'].str.split(' ').str[0]

In [69]:
df

,Unnamed: 0,motherBoard,MRP
0,0,ASRock B450M-HDV Motherboard,"₹5,678"
1,1,ZEBRONICS ZEB-H81M2 with PCIE M.2 / NVMe Slot ...,"₹2,449"
2,2,ZEBRONICS H61-i5-4GB-PFAN-COMBO Motherboard,"₹3,799"
3,3,ZEBRONICS Zeb H61M2 Motherboard with M2 slot L...,"₹2,299"
4,4,ASUS PRIME H610M-E D4 Motherboard,"₹8,950"
...,...,...,...
1435,1435,Raspberry Pi Zero 2W A-Tiny Computer Motherboard,"₹2,499"
1436,1436,Frantech Frontech_H61 Motherboard,"₹3,299"
1437,1437,ZEBRONICS H55+128GB SATA SSD + 4GB starlite DD...,"₹6,900"
1438,1438,nVIDIA Jetson Nano 4GB Developer Kit - B01 Mod...,"₹19,500"


In [65]:
#sql export
from sqlalchemy import create_engine

#Placeholder localhost login
mysql_config = {
    'user': 'pycharm',
    'password': 'testtest-123',
    'host': 'localhost',
    'database': 'projectcpu'
}

engine = create_engine(
    f"mysql+mysqlconnector://{mysql_config['user']}:{mysql_config['password']}@{mysql_config['host']}/{mysql_config['database']}")

#Use only third of the data atm
df.to_sql(name='mobo', con=engine, if_exists='append', index=False)



308

In [67]:
#quick test query
from sqlalchemy import text
connection = engine.connect()

query = text("select * from mobo")
result = connection.execute(query)

counter = 0
for row in result:
    counter+=1
    print(row)
    
print(counter)
result.close()
connection.close()

(0, 'ASRock B450M-HDV Motherboard', 68.26, 'ASRock')
(1, 'ZEBRONICS ZEB-H81M2 with PCIE M.2 / NVMe Slot Motherboard', 29.44, 'ZEBRONICS')
(2, 'ZEBRONICS H61-i5-4GB-PFAN-COMBO Motherboard', 45.67, 'ZEBRONICS')
(3, 'ZEBRONICS Zeb H61M2 Motherboard with M2 slot LGA 1155 Socket Motherboard', 27.64, 'ZEBRONICS')
(4, 'ASUS PRIME H610M-E D4 Motherboard', 107.6, 'ASUS')
(5, 'MSI B550M PRO-VDH WIFI Motherboard', 156.27, 'MSI')
(6, 'ZEBRONICS Zeb-H61M2 Motherboard', 27.64, 'ZEBRONICS')
(7, 'Ant Value G41MAD3 Motherboard', 25.98, 'Ant')
(8, 'ASUS PRIME B550M-A WIFI II Motherboard', 168.3, 'ASUS')
(9, 'ASUS PRIME A320M-K Motherboard', 72.12, 'ASUS')
(10, 'ZEBRONICS ZEB-H61 Motherboard', 27.64, 'ZEBRONICS')
(11, 'ASUS Prime-B450M-A-II Motherboard', 108.19, 'ASUS')
(12, 'ZEBRONICS H61 Motherboard', 25.29, 'ZEBRONICS')
(13, 'ASRock B550M Pro4 Motherboard', 112.87, 'ASRock')
(14, 'GIGABYTE A520M K Motherboard', 70.92, 'GIGABYTE')
(15, 'ASUS ROG STRIX Z590-E Gaming Wi-Fi Motherboard', 402.94, 'ASUS')
(